In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#loading Libraries
import numpy as np
import pandas as pd

In [3]:
#load the preprocessed data 
with open("/content/drive/MyDrive/Cell_Phones_and_Accessories_5.json", "r") as f:
    data = pd.read_json(f, orient="records", lines=True)
#data=pd.read_csv('/content/drive/My Drive/preprocessed_data.csv',nrows=100000) 

In [4]:
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


# Featurization using Word2Vec

In [5]:
## text preprocessing modules
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec
import re
import nltk

In [6]:
### Take the reviews text as a list of elements and then perform the basic preprocessing 
reviews= list(data['reviewText'])
text=[]
for r in reviews:
  text.append(remove_stopwords(r))


In [7]:
### Basic text preprocessing where we remove special characters, any tags etc
#### Removal of stopwords
#### Removal of tags
#### Removal of special characters
#### Removal of email id
#### Removal of numbers
text=[]
for r in reviews:
  r = re.sub(r'<.*?>',"",r)
  r = re.sub(r'\S+@\S+\s','',r)
  r = re.sub(r'\d+','',r)
  r = re.sub(r'[#$!\*\)\(\\%:;,\'_-]','',r)
  text.append(remove_stopwords(r))

In [8]:
### Tokenisation
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=5000,lower=True,oov_token='UNK')

In [9]:
X_train= text[: int(len(text)*0.7)]
X_test = text[int(len(text)*0.7): int(len(text)*0.9)]
X_cv = text[int(len(text)*0.9):]


In [10]:
#Y train, val y test splitting 
Y=list(data['overall'])
y_train= Y[: int(len(text)*0.7)]
y_test = Y[int(len(text)*0.7):int(len(text)*0.9)]
y_cv = Y[int(len(text)*0.9):]

print(len(y_train),len(y_test), len(y_cv))

136107 38888 19444


In [11]:
print(len(X_train), len(X_test), len(X_cv))

136107 38888 19444


In [12]:
# Train your own Word2Vec model using your own text corpus
i=0
train_list_of_sentance=[]
for sentance in X_train:
    train_list_of_sentance.append(sentance.split())

In [13]:
# Train your own Word2Vec model using your own text corpus for cross_validation data
i=0
cv_list_of_sentance=[]
for sentance in X_cv:
    cv_list_of_sentance.append(sentance.split())

In [14]:
# Train your own Word2Vec model using your own text corpus for test data
i=0
test_list_of_sentance=[]
for sentance in X_test:
    test_list_of_sentance.append(sentance.split())

In [15]:
print(len(train_list_of_sentance))
print(len(cv_list_of_sentance))
print(len(test_list_of_sentance))

136107
19444
38888


# Using google's word2vec

In [16]:
# Using Google Word2Vectors
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
# min_count = 5 considers only words that occured atleast 5 times
train_w2v_model=Word2Vec(train_list_of_sentance,min_count=5,size=100, workers=4)

In [17]:
#train_data
train_w2v_words = list(train_w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(train_w2v_words))
print("sample words ", train_w2v_words[0:50])

number of words that occured minimum 5 times  30901
sample words  ['They', 'look', 'good', 'stick', 'I', 'dont', 'like', 'rounded', 'shape', 'bumping', 'Siri', 'kept', 'popping', 'irritating.', 'wont', 'buy', 'product', 'These', 'stickers', 'work', 'review', 'says', 'do.', 'great', 'stay', 'phone.', 'super', 'stylish', 'share', 'sister.', 'awesome', 'phone', 'far', 'year', 'CAN', 'YOU', 'BELIEVE', 'THAT', 'ONE', 'YEAR', 'Great', 'quality', 'Item', 'arrived', 'time', 'perfect', 'condition.', 'However', 'ordered', 'buttons']


In [18]:
#cross_validation_data
cv_w2v_model=Word2Vec(cv_list_of_sentance,min_count=5,size=100, workers=4)
cv_w2v_words = list(cv_w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(cv_w2v_words))
print("sample words ", cv_w2v_words[0:50])

number of words that occured minimum 5 times  13513
sample words  ['Not', 'sure', 'protect', 'phone', 'drop', 'feels', 'nice', 'snug', 'phone.', 'I', 'bulkier', 'case', 'much.', 'This', 'better.', 'fits', 'perfectly', 'Nexus', 'room', 'all.', 'It', 'charge', 'wireless', 'charger', 'well.', 'sharp', 'edges', 'comfortable', 'hand.', 'And', 'great', 'case.', 'Better', 'nexus', 'fit', 'Love', 'products.', 'No', 'cracking', 'far', 'BUT', 'seams', 'pinky', 'encounter', 'near', 'port', 'harm', 'snag', 'annoying', 'dont']


In [19]:
#test_data
test_w2v_model=Word2Vec(test_list_of_sentance,min_count=5,size=100, workers=4)  
test_w2v_words = list(test_w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(test_w2v_words))
print("sample words ", test_w2v_words[0:50])

number of words that occured minimum 5 times  16588
sample words  ['This', 'case', 'nice', 'dollar', 'youre', 'going', 'Do', 'expect', 'amazing', 'gold', 'brand', 'material', 'paying', 'so.', 'If', 'dont', 'want', 'like', 'high', 'prices', 'probably', 'good', 'you.', 'The', 'magnet', 'little', 'closed', 'inside', 'kind', 'soft.', 'Overall', 'bad', 'product', 'cost', 'THis', 'perfect', 'iphone', 'great', 'fit', 'looks', 'too.', 'It', 'way', 'better', 'getting', 'expensive', 'ones', 'carrier.', 'I', 'case.']


# Converting text into vectors using Avg W2V

In [20]:
# average Word2Vec (train)
# compute average word2vec for each review.
train_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in train_list_of_sentance: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length 100, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in train_w2v_words:
            vec = train_w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    train_vectors.append(sent_vec)
train_vectors = np.array(train_vectors)
print(train_vectors.shape)
print(train_vectors[0])

(136107, 100)
[ 0.57097848  0.24361564  0.23096872 -0.11885987  0.408756   -0.11845835
  0.18787493  0.44578827  0.6537884  -0.27505032 -1.06347573 -0.26731543
 -0.04897028  0.36155277  0.20636253 -0.27400332 -0.27254389  0.14585912
  0.32762919 -0.04813974 -0.40939541  0.43081103  0.39082643 -0.89792542
 -0.36027618 -0.19892159  0.06931548 -0.05733299 -0.17057233 -0.18786373
 -0.9567457   0.6271354  -0.85650174 -0.71612391  0.76705054  0.01857592
 -0.28593277 -0.30753889  0.5394285  -0.05903247  0.10104866 -0.04702524
  0.89047091 -0.20596848 -0.56546688  0.04375344  0.55835804 -0.31575615
  0.51878268 -0.28976496  0.32791098  0.01166676  0.12300571 -0.22258025
  0.62195042  0.48492294 -0.34841706  0.63148413  0.06820225 -0.26669993
 -0.37017614 -0.12358348 -0.38654213  0.11264476  0.22099719  0.03589316
  0.29316744  0.00626232  0.36734739 -0.37041313 -0.04766933 -0.48768047
  0.148275    0.86316781  0.69430728 -0.49254856 -0.72178579  0.20121414
 -0.11516084 -0.13969396  0.12508489 

In [21]:
# average Word2Vec (for validation data)
# compute average word2vec for each review.
cv_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in cv_list_of_sentance: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in cv_w2v_words:
            vec = cv_w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    cv_vectors.append(sent_vec)
cv_vectors = np.array(cv_vectors)
print(cv_vectors.shape)
print(cv_vectors[0])

(19444, 100)
[ 0.41713728  0.50905562  0.41746652 -0.1244351   0.35360124 -0.30348286
 -0.59427893  0.90974214 -0.27630282  0.04204035  0.07842532  0.2112373
  0.2051717   0.38089061  0.5591423   0.43226828  0.02357326 -0.33672818
  0.20681069  0.82463864 -0.5261638   0.56692636  0.04158936 -0.06408776
 -0.07141631  0.0889906  -0.12510539 -0.54679686 -0.33733969 -0.26645028
  0.21769933 -0.06334062 -0.11537957 -0.21262521  0.71250436 -0.74038041
 -0.5838121   0.03596484 -0.11266859 -0.09826147  0.38498385 -0.16866102
  0.36219312  0.31576002 -0.45493094  0.30004026 -0.51901997  0.31056083
 -0.08126357 -0.1690063   0.07017675 -0.59075461 -0.24910519 -0.5607725
 -0.21460338  0.78566102 -0.74657888  0.78417388  0.58608291 -0.68880415
 -0.80335677  0.74428119  0.18470489  0.28942793 -0.42955814  0.2387385
 -0.11341193 -0.58660254 -0.26174166 -0.06505774  0.86984989  0.36078771
 -0.19212712  0.49322365  0.01479115 -0.27584367  0.16374894 -0.17890404
  0.47983456 -0.38616195 -0.20792042  0.1

In [22]:
# average Word2Vec (test)
# compute average word2vec for each review.
test_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in test_list_of_sentance: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in test_w2v_words:
            vec = test_w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    test_vectors.append(sent_vec)
test_vectors = np.array(test_vectors)
print(test_vectors.shape)
print(test_vectors[0])

(38888, 100)
[ 0.003871    0.16939315  0.24115582 -0.28731937 -0.01155992 -0.29911234
  0.10702542  0.37285543 -0.95691155  0.05670592 -0.25972376 -0.06119378
 -0.84479733  0.51935808  0.08199053 -0.19234736 -0.23693585 -0.1762489
  0.28240125  0.53959732 -0.08013337  0.61090956  0.69607299 -0.16393586
 -0.41216821  0.26745137  0.71569414 -0.93554069 -0.733216    0.13203837
 -0.01754943  0.32060953 -0.25490542 -0.45131316  0.11151119 -0.16834358
 -0.32664638  0.00171321 -0.4720588  -0.23361284  0.41189478 -0.17534936
  0.40374226 -0.06505203  0.12204036  0.09621765 -0.23065021  0.212929
  0.16796    -0.08512775  0.19766052 -0.28375505  0.21533479  0.00372612
 -0.15172233  0.39014444 -0.60392491 -0.05153138 -0.33373492 -0.12865362
  0.05347615  0.41887112 -0.21368067  0.4470589  -0.37235609  0.16001248
 -0.511536   -0.57022134  0.14424778 -0.23305538  0.41517297  0.10824225
 -0.15393791  0.59670382  0.23416406 -0.21985858  0.07444178 -0.38938865
 -0.16681788 -0.06194936  0.39478787  0.1

In [23]:
avgw2v_train= np.array(train_vectors)
avgw2v_cv= np.array(cv_vectors)
avgw2v_test= np.array(test_vectors)

#Model building

In [24]:
#LOGISTIC REGRESSION 
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class='multinomial')
lr.fit(avgw2v_train, y_train)
val_pred = lr.predict(avgw2v_cv)
test_pred = lr.predict(avgw2v_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [25]:
from sklearn.metrics import accuracy_score
print("Validation accuracy:", accuracy_score(val_pred,y_cv)*100)

Validation accuracy: 60.08023040526641


In [26]:
from sklearn.metrics import accuracy_score
print("Test accuracy:", accuracy_score(test_pred,y_test)*100)

Test accuracy: 56.42614688335733


#Using GLOVE model (50D)

In [29]:
embeddings_index50= dict()
glove50= open('/content/drive/MyDrive/glove.6B.200d.txt','r',encoding='utf-8')
for line in glove50:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index50[word] = coefs

glove50.close()

In [30]:
len(embeddings_index50)

400000

In [68]:
### Create an embedding matrix for the vocabulary created for the reviews
vocab50= len(tokenizer.word_index)+1
embedding_matrix50 = np.zeros((vocab50, 200)) 
for word, i in tokenizer.word_index.items():
    embedding_vector50 = embeddings_index50.get(word)
    if embedding_vector50 is not None:
        embedding_matrix50[i] = embedding_vector50

In [69]:
#textstosequence allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...
train_indices50=tokenizer.texts_to_sequences(X_train)
val_indices50=tokenizer.texts_to_sequences(X_cv)
test_indices50=tokenizer.texts_to_sequences(X_test)

In [70]:
#Padding (limit is 100)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length=100
train_indices50=pad_sequences(train_indices50,maxlen=max_length,padding='post')
test_indices50=pad_sequences(test_indices50,maxlen=max_length,padding='post')
val_indices50=pad_sequences(test_indices50,maxlen=max_length,padding='post')

In [71]:
train_50=np.asarray(train_indices50)
train_y=np.asarray(y_train)
val_indices50=np.asarray(val_indices50)
val_y=np.asarray(y_cv)
test_indices50=np.asarray(test_indices50)
test_y=np.asarray(y_test)

In [72]:
##########     MODEL BUILDING (LOGISTIC REGRESSION)
lr50 = LogisticRegression(multi_class='multinomial')
lr50.fit(train_indices50,train_y)
val_pred_lr50 = lr50.predict(val_indices50)
test_pred_lr50 = lr50.predict(test_indices50)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [73]:
# Final evaluation of the model
print("Test accuracy:", accuracy_score(test_pred_lr50,y_test)*100)

Test accuracy: 57.32616745525612


#Using GLOVE model (100D)

In [37]:
#Tokenisation on train, test and val
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
tokenizer.fit_on_texts(X_cv)

In [40]:
#### Now we need embeddings for these words  
## Get the glove vectors
embeddings_index= dict()
glove= open('/content/drive/MyDrive/glove.6B.100d.txt','r',encoding='utf-8')
for line in glove:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

glove.close()


In [41]:
word

'sandberger'

In [42]:
tokenizer.word_index


{'UNK': 1,
 'i': 2,
 'phone': 3,
 'case': 4,
 'the': 5,
 'it': 6,
 'like': 7,
 'this': 8,
 'great': 9,
 'use': 10,
 'screen': 11,
 'good': 12,
 'battery': 13,
 'iphone': 14,
 'charge': 15,
 'charger': 16,
 'product': 17,
 'time': 18,
 'dont': 19,
 'works': 20,
 's': 21,
 'little': 22,
 'nice': 23,
 'price': 24,
 'love': 25,
 'work': 26,
 'charging': 27,
 'quality': 28,
 'fit': 29,
 'im': 30,
 'easy': 31,
 'usb': 32,
 'device': 33,
 'its': 34,
 'protector': 35,
 'ive': 36,
 'power': 37,
 'got': 38,
 'better': 39,
 'bought': 40,
 'doesnt': 41,
 'cable': 42,
 'recommend': 43,
 'need': 44,
 'fits': 45,
 'if': 46,
 'cover': 47,
 'looks': 48,
 'buy': 49,
 'protection': 50,
 'cases': 51,
 'new': 52,
 'car': 53,
 'way': 54,
 'sound': 55,
 'my': 56,
 'phones': 57,
 'you': 58,
 'well': 59,
 'galaxy': 60,
 'thing': 61,
 'bluetooth': 62,
 'want': 63,
 'hard': 64,
 'but': 65,
 'so': 66,
 'devices': 67,
 'a': 68,
 'pretty': 69,
 'long': 70,
 'one': 71,
 'headset': 72,
 'samsung': 73,
 'bit': 74,
 'p

In [43]:
len(tokenizer.word_index)

108086

In [44]:
len(embeddings_index)

400000

In [45]:
type(embeddings_index)

dict

In [46]:
### Create an embedding matrix for the vocabulary created for the reviews 
vocab= len(tokenizer.word_index)+1
embedding_matrix = np.zeros((vocab, )) 
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [47]:
embedding_matrix.shape

(108087, 100)

In [48]:
#textstosequence allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...
train_indices=tokenizer.texts_to_sequences(X_train)
val_indices=tokenizer.texts_to_sequences(X_cv)
test_indices=tokenizer.texts_to_sequences(X_test)

In [50]:
## Padding (we are limiting the number of words to 64)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length=64
train_indices=pad_sequences(train_indices,maxlen=max_length,padding='post')
test_indices=pad_sequences(test_indices,maxlen=max_length,padding='post')
val_indices=pad_sequences(test_indices,maxlen=max_length,padding='post')

In [51]:
train_indices=np.asarray(train_indices)
train_y=np.asarray(y_train)
val_indices=np.asarray(val_indices)
val_y=np.asarray(y_cv)
test_indices=np.asarray(test_indices)
test_y=np.asarray(y_test)


In [52]:
##########     MODEL BUILDING (LOGISTIC REGRESSION)
lr2 = LogisticRegression(multi_class='multinomial')
lr2.fit(train_indices,train_y)
val_pred_lr2 = lr2.predict(val_indices)
test_pred_lr2 = lr2.predict(test_indices)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [53]:
# Final evaluation of the model
print("Test accuracy:", accuracy_score(test_pred_lr2,y_test)*100)


Test accuracy: 57.4444558732771


#Using GLOVE 200D model

In [54]:
embeddings_index200= dict()
glove200= open('/content/drive/MyDrive/glove.6B.200d.txt','r',encoding='utf-8')
for line in glove200:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index200[word] = coefs

glove200.close()

In [58]:
### Create an embedding matrix for the vocabulary created for the reviews 
vocab= len(tokenizer.word_index)+1
embedding_matrix200 = np.zeros((vocab, 100)) 
for word, i in tokenizer.word_index.items():
    embedding_vector200 = embeddings_index.get(word)
    if embedding_vector200 is not None:
        embedding_matrix200[i] = embedding_vector200

In [59]:
embedding_matrix200.shape

(108087, 100)

In [60]:
#textstosequence allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...
train_indices200=tokenizer.texts_to_sequences(X_train)
val_indices200=tokenizer.texts_to_sequences(X_cv)
test_indices200=tokenizer.texts_to_sequences(X_test)

In [61]:
## Padding (we are limiting the number of words to 100)
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length=100
train_indices200=pad_sequences(train_indices,maxlen=max_length,padding='post')
test_indices200=pad_sequences(test_indices,maxlen=max_length,padding='post')
val_indices200=pad_sequences(test_indices,maxlen=max_length,padding='post')

In [62]:
train_indices200=np.asarray(train_indices200)
train_y=np.asarray(y_train)
val_indices200=np.asarray(val_indices200)
val_y=np.asarray(y_cv)
test_indices200=np.asarray(test_indices200)
test_y=np.asarray(y_test)


In [63]:
##########     MODEL BUILDING (LOGISTIC REGRESSION)
lr3 = LogisticRegression(multi_class='multinomial')
lr3.fit(train_indices200,train_y)
val_pred_lr3 = lr3.predict(val_indices200)
test_pred_lr3 = lr3.predict(test_indices200)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [64]:
# Final evaluation of the model
print("Test accuracy:", accuracy_score(test_pred_lr3,y_test)*100)

Test accuracy: 57.43674141123226
